In [1]:
import os
print(os.getcwd())

c:\Users\UTENTE\Git Repos\Chicken-Disease-Classification--Project\research


In [2]:
os.chdir("../")
os.getcwd()

'c:\\Users\\UTENTE\\Git Repos\\Chicken-Disease-Classification--Project'

In [3]:
from pathlib import Path
from dataclasses import dataclass


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [4]:
from src.cnnClassifier.utils.common import read_yaml, create_directories
from src.cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
import tensorflow as tf
from src.cnnClassifier import logger

In [5]:

class ConfigurationManager:

    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
        config = self.config.prepare_callbacks
        model_ckpt_callback_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([
            Path(model_ckpt_callback_dir),
            Path(config.tensorboard_root_log_dir),
        ])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,
                                     "Chicken-fecal-images")
        create_directories([training.root_dir])

        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augmentation = params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE
        )

        return training_config
    


In [6]:
import time
import tensorflow as tf

In [7]:
class PrepareCallback:
    def __init__(self, config:PrepareCallbackConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir,
                     f"tb_logs_at_{timestamp}")
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [8]:
import tensorflow as tf

In [11]:
class Training:

    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        logger.info(f"fetching model from {self.config.updated_base_model_path}")
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )


    # generating train and test data (features and ohe labels)
    def train_valid_generator(self):
        logger.info("Creating train_generator and test_generator...")

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            logger.info("Augmentation applied to training data...")
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )


    @staticmethod
    def save_model(model: tf.keras.Model, path:Path):
        logger.info(f"Saving trained model to {path}")
        model.save(path)



    def train(self, callback_list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size


        logger.info("Starting training ...")
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )
        logger.info("Training finished !!!")


        self.save_model(path=self.config.trained_model_path,
                        model=self.model)


In [13]:
try:
    config = ConfigurationManager()
    prepare_callback_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callback_config)    
    callbacks_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list=callbacks_list)

except Exception as e:
    raise e

[2025-05-14 17:07:56,850: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-14 17:07:56,850: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-14 17:07:56,850: INFO: common: created directory at: artifacts]
[2025-05-14 17:07:56,850: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-05-14 17:07:56,850: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-05-14 17:07:56,866: INFO: common: created directory at: artifacts/training]
[2025-05-14 17:07:56,866: INFO: 2504865915: fetching model from artifacts\prepare_base_model\base_model_updated.keras]
[2025-05-14 17:07:57,200: INFO: 2504865915: Creating train_generator and test_generator...]
Found 78 images belonging to 2 classes.
[2025-05-14 17:07:57,215: INFO: 2504865915: Augmentation applied to training data...]
Found 312 images belonging to 2 classes.
[2025-05-14 17:07:57,238: INFO: 2504865915: Starting training ...]
19/19 ━━